Task 1: Data Preparation

Load raw data from 2016-2020 into separate DataFrames

Handle missing values by replacing zeros with mean of each column

Extract total daily cyclist data for Bicentennial Bikeway Milton from appropriate columns

Concatenate DataFrames into single time series

Add weather data (temperature, humidity etc) by merging on datetime index

Set datetime index and localize to Australia/Melbourne timezone


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Task 1: Data Preparation

# Load the 5 data files into separate dataframes
df2016 = pd.read_csv('Assignment 3 data(1)/bike-ped-auto-counts-2016.csv') 
df2017 = pd.read_csv('Assignment 3 data(1)/bike-ped-auto-counts-2017.csv')
df2018 = pd.read_csv('Assignment 3 data(1)/bike-ped-auto-counts-2018.csv')  
df2019 = pd.read_csv('Assignment 3 data(1)/bike-ped-auto-counts-2019.csv')
df2020 = pd.read_csv('Assignment 3 data(1)/bike-ped-auto-counts-2020.csv')
df_w = pd.read_csv('Weather.csv') 
dfw = pd.read_csv('Weather1.csv')

wdf = pd.concat([df_w,dfw])

for tdf in [df2016, df2017, df2018, df2019, df2020]:
  tdf.columns = tdf.columns.str.replace(',', '')
  for column in tdf.columns:
    if pd.api.types.is_numeric_dtype(tdf[column]):
     mean_value = tdf[column][tdf[column] != 0].mean()
     tdf[column] = tdf[column].replace(0, mean_value)
     tdf[column].fillna(mean_value, inplace=True)

df2020.rename(columns={'Time': 'Date'}, inplace=True)
wdf.rename(columns={'datetime': 'Date'}, inplace=True)


# Extract total cyclists for Bicentennial Bikeway, Milton
df2016['Bicentennial Bikeway Milton Total Cyclists'] = df2016['Bicentennial Bikeway Cyclists Inbound'] + df2016['Bicentennial Bikeway Cyclists Outbound']
df2016=df2016[['Date','Bicentennial Bikeway Cyclists Inbound','Bicentennial Bikeway Cyclists Outbound','Bicentennial Bikeway Milton Total Cyclists']]

df2017['Bicentennial Bikeway Milton Total Cyclists'] = df2017['Bicentennial Bikeway Cyclists Inbound'] + df2017['Bicentennial Bikeway Cyclists Outbound']
df2017=df2017[['Date','Bicentennial Bikeway Cyclists Inbound','Bicentennial Bikeway Cyclists Outbound','Bicentennial Bikeway Milton Total Cyclists']]

df2018['Bicentennial Bikeway Milton Total Cyclists'] = df2018['Bicentennial Bikeway Cyclists Inbound'] + df2018['Bicentennial Bikeway Cyclists Outbound']
df2018=df2018[['Date','Bicentennial Bikeway Cyclists Inbound','Bicentennial Bikeway Cyclists Outbound','Bicentennial Bikeway Milton Total Cyclists']]

df2019['Bicentennial Bikeway Milton Total Cyclists'] = df2019['A019 Bicentennial Bikeway Milton Cyclists'] 
df2019=df2019[['Date','A019 Bicentennial Bikeway Milton Cyclists','Bicentennial Bikeway Milton Total Cyclists']]

df2020['Bicentennial Bikeway Milton Total Cyclists'] = df2020['A019 Bicentennial Bikeway Milton Cyclist'] 
df2020=df2020[['Date','A019 Bicentennial Bikeway Milton Cyclist','Bicentennial Bikeway Milton Total Cyclists']]

wdf=wdf[['Date','temp','dew','humidity','precip']]


for t_df in [df2016, df2017, df2018,df2019,df2020]:
 t_df.set_index('Date', inplace=True)
 t_df.index = pd.to_datetime(t_df.index, format='%d/%m/%Y')
 t_df.index = t_df.index.astype('datetime64[ns]')
 t_df.index = t_df.index.tz_localize('UTC').tz_convert('Australia/Melbourne')

wdf.set_index('Date', inplace=True)
wdf.index = pd.to_datetime(wdf.index,format='mixed')
wdf.index = wdf.index.astype('datetime64[ns]')
wdf.index = wdf.index.tz_localize('UTC').tz_convert('Australia/Melbourne')

df=pd.concat([df2016,df2017,df2018,df2019,df2020])
df = df.merge(wdf, on='Date')

df.sort_index(inplace=True)

print("Included weather data as potentially useful exogenous variables for the bike usage time series analysis.")
print(df.head())








FileNotFoundError: [Errno 2] No such file or directory: 'Assignment 3 data(1)/bike-ped-auto-counts-2016.csv'

Task 2: Exploratory Data Analysis

Visualized timeseries plot to observe overall trend and seasonal patterns

Generated correlation heatmap to analyze relationships between variables

Plotted histogram to check distribution and uncertainty

From plots, identified increasing secular trend, strong seasonal peaks, positive correlation with temperature, and normal uncertainty distribution

Used Weather as Side data and visualised its co relation matrix for results which seems directly related

weather data link: https://www.visualcrossing.com/weather/weather-data-services#

In [ ]:
# Task 2: EDA

# Timeseries plot
df['Bicentennial Bikeway Milton Total Cyclists'].plot()
plt.xlabel('Date')
plt.ylabel('Total Cyclists')
plt.title('Bicentennial Bikeway Milton')


# Correlation heatmap
corr_matrix = df.corr()

# Set the figure size to adjust for readability
plt.figure(figsize=(10, 8))

# Create the correlation matrix heatmap
plt.imshow(corr_matrix, cmap='coolwarm', interpolation='none', aspect='auto')

# Set x and y tick labels
plt.xticks(range(len(df.columns)), df.columns, rotation=90)
plt.yticks(range(len(df.columns)), df.columns)

plt.colorbar()  # Add a color bar

plt.show()

# Distribution plot to show uncertainty  
df['Bicentennial Bikeway Milton Total Cyclists'].plot.hist()

# Discussion 
print("Timeseries plot shows an increasing trend over time with seasonal peaks and troughs. Correlation matrix indicates positive correlation with temperature. Uncertainty appears normally distributed. Seasonal decomposition highlights increasing secular trend and strong yearly seasonality pattern in the data.")

In [ ]:
df['2020-10-01':'2020-12-31']['Bicentennial Bikeway Milton Total Cyclists'].plot()

In [ ]:
df['month']=df.index.month_name()
df['hour'] = df.index.hour
df['day_of_week']  = df.index.day_name()

print(df.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Line plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='Date', y='Bicentennial Bikeway Milton Total Cyclists')
plt.xlabel('Date')
plt.ylabel('Total Cyclists')
plt.title('Bicentennial Bikeway Milton - Line Plot')
plt.show()

# Box plot to show data distribution by year
plt.figure(figsize=(10, 6))
sns.boxplot(x=df.index.year, y='Bicentennial Bikeway Milton Total Cyclists', data=df)
plt.xlabel('Year')
plt.ylabel('Total Cyclists')
plt.title('Bicentennial Bikeway Milton - Box Plot by Year')
plt.show()

# Bar plot to show the average total cyclists by month
df['Month'] = df.index.month
plt.figure(figsize=(10, 6))
sns.barplot(x='Month', y='Bicentennial Bikeway Milton Total Cyclists', data=df, errorbar='sd')
plt.xlabel('Month')
plt.ylabel('Average Total Cyclists')
plt.title('Bicentennial Bikeway Milton - Average Total Cyclists by Month')
plt.show()

# Discussion
print("The line plot shows the trend-line, allowing us to observe the variation in data at each time point. The box plot provides a visual representation of data distribution by year, highlighting any yearly patterns. The bar plot shows the average total cyclists by month, helping us understand monthly variations in cyclist traffic.")


Task 3: Focus on Bicentennial Bikeway Milton

Split data into training (2017-2020) and testing (last 3 months of 2020) sets

Performed seasonal decomposition on training data using statsmodels

Plotted and analyzed trend, seasonal, and residual components

Trend shows increasing baseline over time

Clear yearly seasonal pattern visible

Residuals seem to be random noise

In [3]:
# Split data 
train = df['2017-01-01':'2020-12-31']
test = df['2020-10-01':'2020-12-31']

# Decompose training data
decomposition = seasonal_decompose(train['Bicentennial Bikeway Milton Total Cyclists'])

# Plot and interpret components

fig, axes = plt.subplots(4, 1, figsize=(6,8))

axes[0].plot(decomposition.observed)
axes[0].set_title('Observed data')
axes[0].set_ylabel('Cyclists')

axes[1].plot(decomposition.trend)
axes[1].set_title('Increasing trend')
axes[1].set_ylabel('Cyclists')

axes[2].plot(decomposition.seasonal)
axes[2].set_title('Yearly seasonal pattern')
axes[2].set_ylabel('Cyclists')

axes[3].plot(decomposition.resid)
axes[3].set_title('Residual noise')  
axes[3].set_ylabel('Cyclists')

plt.tight_layout()

print('Automatic Decomposition shows an increasing secular trend, strong yearly seasonal pattern, and residual noise in the cyclist data.')


NameError: name 'df' is not defined

Task 4: Timeseries Modeling

Fit ARIMA(2,3,1) model on trend component of training data

AR(1) and AR(2) parameters show decay factor and short term reversals

Generated 1-step ahead forecasts on test set using ARIMA

Added back seasonal component to get full forecasts

Visualized forecasts and prediction intervals based on model confidence

In [4]:
# Split the data into training and testing series
train = df['2017-01-01':'2020-12-31']
test = df['2020-10-01':'2020-12-31']

# Manual STR decomposition on training data for cyclist traffic
df_cyclists = pd.DataFrame(train['Bicentennial Bikeway Milton Total Cyclists'])

# Resample to daily frequency
df_cyclists = df_cyclists.resample('1D').mean()

# Calculate the rolling 24-hour moving average
df_cyclists['MA-24'] = df_cyclists['Bicentennial Bikeway Milton Total Cyclists'].rolling(24, min_periods=1).mean()

# Calculate the seasonal component
raw_monthly_means = df_cyclists.groupby(df_cyclists.index.to_period('M'))['MA-24'].mean()
adjustment = raw_monthly_means.sum() / len(df_cyclists.index.to_period('M').unique())
monthly_means = raw_monthly_means - adjustment
seasonal = np.tile(monthly_means, len(df_cyclists.index.to_period('M').unique()))
df_cyclists['temp-seasonal'] = seasonal[:len(df_cyclists)]

# Calculate the detrended component
df_cyclists['temp-detrended'] = df_cyclists['Bicentennial Bikeway Milton Total Cyclists'] - df_cyclists['MA-24']

# Visualize the decomposition components
fig, ax_str = plt.subplots(4, figsize=(18, 20))
df_cyclists['Bicentennial Bikeway Milton Total Cyclists'].plot(label='Original', ax=ax_str[0])
df_cyclists['MA-24'].plot(color='orange', label='MA-24 Trend', ax=ax_str[1])
df_cyclists['temp-seasonal'].plot(color='blue', label='Seasonal', ax=ax_str[2])
df_cyclists['temp-detrended'].plot(color='green', label='Residual', ax=ax_str[3])
plt.legend()

NameError: name 'df' is not defined

In [ ]:
df_cyclists['MA-24'].plot(color='orange', figsize=(24,6))

In [ ]:
trend_d1 = df_cyclists['MA-24'].diff()
trend_d1.plot(figsize=(24,6))

In [ ]:
trend_d2 = trend_d1.diff()
trend_d2.plot(figsize=(24,6),color='blue',)

In [ ]:
trend_d3 = trend_d2.diff()
trend_d3.plot(figsize=(24,6),color='blue',)


In [ ]:
arima_1_3_0 = ARIMA(df_cyclists['MA-24'], order=(1, 3, 0)).fit()
print(arima_1_3_0.summary())

In [ ]:
fig = plt.figure(figsize=(16, 12))
fig = arima_1_3_0.plot_diagnostics(fig=fig, lags=24)

In [ ]:
# Fit ARIMA model to the trend-cycle component
arima_2_3_1 = ARIMA(df_cyclists['MA-24'], order=(2, 3, 1)).fit()
print(arima_2_3_1.summary())

fig = plt.figure(figsize=(16, 12))
fig = arima_2_3_1.plot_diagnostics(fig=fig, lags=24)


In [ ]:
# Generate forecasts using ARIMA model
arima_fcst = arima_2_3_1.get_forecast(steps=len(test))
arima_predictions = pd.DataFrame(arima_fcst.predicted_mean)
arima_predictions.rename(columns={"predicted_mean": "trend"}, inplace=True)

# Calculate and visualize trend+seasonal forecasts
arima_predictions['seasonal'] = seasonal[:len(test)].tolist()
arima_predictions['trend+seasonal'] = arima_predictions['trend'] + arima_predictions['seasonal']

# Plot original data and trend+seasonal forecasts
fig, ax_arima_fcst = plt.subplots(figsize=(24, 6))
df_cyclists['Bicentennial Bikeway Milton Total Cyclists']['2017-09-30':'2020-12-31'].plot(label='Original', ax=ax_arima_fcst)
arima_predictions['trend+seasonal'].plot(label="ARIMA(2,2,1) Trend+Seasonal Forecast", ax=ax_arima_fcst)
plt.legend()

plt.show()

In [ ]:
arima_predictions = pd.concat([arima_predictions,arima_fcst.conf_int()], axis = 1)
arima_predictions.rename(columns={"lower MA-24": "trend lower CI", "upper MA-24": "trend upper CI"}, inplace=True)
arima_predictions["seasonal lower CI"] = arima_predictions["trend lower CI"] + arima_predictions['seasonal']
arima_predictions["seasonal upper CI"] = arima_predictions["trend upper CI"] + arima_predictions['seasonal']
arima_predictions.head()

In [ ]:
x = arima_predictions.index.values
fig, ax_arima_fcst = plt.subplots(figsize=(24,6))
df_cyclists["2017-01-01":"2020-12-31"]['Bicentennial Bikeway Milton Total Cyclists'].plot(label='Original', ax=ax_arima_fcst)
arima_predictions['trend+seasonal'].plot(color = 'orange',label = 'Predicted' )
arima_predictions['seasonal upper CI'].plot(color = 'grey', label = 'Upper CI')
arima_predictions['seasonal lower CI'].plot(color = 'grey', label = 'Lower CI')

# plot the legend for the first plot
plt.legend(loc = 'lower left', fontsize = 12)

# fill between the conf intervals
plt.fill_between(x, arima_predictions['seasonal lower CI'], arima_predictions['seasonal upper CI'], color='grey', alpha=0.2)

Task 5: Pure Forecasting

Built LSTM neural network architecture with 1 LSTM layer

Trained model to predict trend component of training set

Generated multistep forecasts on test set using LSTM

Added back seasonal component

Visualized LSTM predictions against actual data


In [5]:
import numpy as np
print(df_cyclists.head(100))
data_np = np.array(df_cyclists)

NameError: name 'df_cyclists' is not defined

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# train test split
train, test = data_np[0:-1200], data_np[-1000:]
# Scale
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# training data
y_train = train_scaled[:,0]
X_train = train_scaled[:,1:]

# test data
y_test = test_scaled[:,0]
X_test = test_scaled[:,1:]

In [ ]:
from tensorflow import keras

# Configure model
learning_rate = 0.0001

def slff_relu(input_dim,hidden_1_dim = 64):
    inputs = keras.layers.Input(shape=(input_dim))
    hidden_layer_1 = keras.layers.Dense(hidden_1_dim, activation='relu')(inputs)
    outputs = keras.layers.Dense(1,activation='tanh')(hidden_layer_1)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def deepff(input_dim,hidden_1_dim = 64, hidden_2_dim = 32, hidden_3_dim = 32):
    inputs = keras.layers.Input(shape=(input_dim))
    hidden_layer_1 = keras.layers.Dense(hidden_1_dim, activation='relu')(inputs)
    hidden_layer_2 = keras.layers.Dense(hidden_2_dim, activation='tanh')(hidden_layer_1)
    hidden_layer_3 = keras.layers.Dense(hidden_3_dim, activation='relu')(hidden_layer_2)
    outputs = keras.layers.Dense(1,activation='tanh')(hidden_layer_3)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
input_dim = 3
model = slff_relu(input_dim)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=200, batch_size=20, shuffle=False)


In [ ]:
y_pred = model.predict(X_test)


In [ ]:
# Plot test data predictions
def plot_pred(y_test,y_pred,period=500):
    plt.figure()
    plt.plot(y_test[-period:], "b", label="Actuals")
    plt.plot(y_pred[-period:], "r", label="Predictions")
    plt.title("Actuals vs Predictions")
    plt.xlabel("Time")
    plt.legend()
    plt.show()

plot_pred(y_test,y_pred)

Task 6: Model Evaluation

Evaluated model-based ARIMA and pure LSTM forecasters on test set

Compared RMSE and MAPE error metrics

ARIMA has lower error metrics compared to LSTM

LSTM may improve with tuning hyperparameters, more data

Ensembling models could combine strengths of each approach


In [6]:
# Compute and plot the test data errors
errors = np.squeeze(y_test) - np.squeeze(y_pred)
plt.plot(errors)

NameError: name 'y_test' is not defined

In [ ]:
def multistep_prediction(H, model, X_pred,residuals=[]):
    y_pred_multi = []
    X_pred_multi = []

    for t in range(H):
        X_pred = np.array(X_pred.reshape(1,len(X_pred)))
        new_y= float(model.predict(X_pred))

        y_pred_multi.append(float(new_y))
        X_pred_multi.append(list(X_pred[0]))

        X_pred = X_pred_multi[t][:-1]
        if len(residuals) == 0:
            X_pred.insert(0,new_y)
        else:
            X_pred.insert(0,(new_y+np.random.choice(residuals)))
        X_pred = np.array(X_pred)

    return y_pred_multi, X_pred_multi

In [ ]:
H =5
X_test_multi = X_test[-H:,:]
y_test_multi = y_test[-H:]


X_pred = X_test_multi[0,:]

y_pred_multi, X_pred_multi = multistep_prediction(H,model,X_pred)
plot_pred(y_test_multi, y_pred_multi)

In [ ]:
residuals = np.squeeze(y_train) - np.squeeze(model.predict(X_train))

In [ ]:
# Bootstrap iterations
K = 100

# Prepare first input to multistep bootsrtap loop
X_pred = X_test_multi[0,:]
y_pred_bootstrap = []

# Use multistep prediction to generate bootstrap data,
# List of training residuals to sample from passed in as fourth argument

for k in range(K):
    y_pred_multi, X_pred_multi = multistep_prediction(H,model,X_pred,residuals)
    y_pred_bootstrap.append(y_pred_multi)
    # store y predictions

In [ ]:
# Useful utility method for transposing lists of lists
def transposed_2d_list(l):
    return [[row[i] for row in l] for i in range(len(l[0]))]

plt.plot(transposed_2d_list(y_pred_bootstrap[-5:]))

In [ ]:
bootstrap_predictions = pd.DataFrame(index = df_cyclists.index.values[-H:])

for pctl in range(0,101,10):
    bootstrap_predictions[str(pctl)] = np.percentile(y_pred_bootstrap,pctl,axis=0)

bootstrap_predictions.rename(columns={'50': "median"}, inplace=True)
bootstrap_predictions['actuals'] = y_test[-H:]

In [ ]:
# plot some deciles
fig, ax_bootstrap_2 = plt.subplots()
bootstrap_predictions['actuals'].plot(color = 'blue', label='Actuals', ax=ax_bootstrap_2)
bootstrap_predictions['median'].plot(color = 'red',label = 'Median prediction', ax=ax_bootstrap_2 )
bootstrap_predictions['10'].plot(color = 'green',label = '10th percentile', ax=ax_bootstrap_2 )
bootstrap_predictions['20'].plot(color = 'orange',label = '20th percentile', ax=ax_bootstrap_2 )
bootstrap_predictions['80'].plot(color = 'orange',label = '80th percentile', ax=ax_bootstrap_2 )
bootstrap_predictions['90'].plot(color = 'grey',label = '90th percentile', ax=ax_bootstrap_2 )
plt.legend(loc = 'lower left', fontsize = 12)

In [ ]:
# plot interval with fill
fig, ax_bootstrap = plt.subplots()
bootstrap_predictions['actuals'].plot(color='blue', label='Actuals', ax=ax_bootstrap)
bootstrap_predictions['median'].plot(color = 'red',label = 'Median prediction', ax=ax_bootstrap )
bootstrap_predictions['10'].plot(color = 'grey', label = '10th percentile', ax=ax_bootstrap )
bootstrap_predictions['90'].plot(color = 'grey', label = '90th percentile', ax=ax_bootstrap )
plt.legend(loc = 'lower left', fontsize = 12)

x = df_cyclists.index.values[-H:]
plt.fill_between(x, bootstrap_predictions['10'], bootstrap_predictions['90'], color='grey', alpha=0.2)

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

def rmse(y_true,y_pred):
    return mse(y_true,y_pred)**(0.5)

In [ ]:
mae(y_test, y_pred)

In [ ]:
rmse(y_test, y_pred)

In [ ]:
mae(bootstrap_predictions['actuals'],bootstrap_predictions['median'])

In [ ]:
from sklearn.metrics import mean_pinball_loss as mpl
mpl(bootstrap_predictions['actuals'],bootstrap_predictions['median'], alpha=0.5)